In [3]:
import pandas as pd
import glob as gl
merge = pd.read_csv('zip_codes.csv')
merge['DOSAGE_UNIT'] = 0
merge = merge.rename(columns={'Zip Code': 'BUYER_ZIP'})
files = sorted(gl.glob('data/*.csv'))
for i, f in enumerate(files):
    df = pd.read_csv(f)
    # Comment next two lines for all years from 2006 to 2012.
    df['TRANSACTION_DATE'] = pd.to_datetime(df['TRANSACTION_DATE'], format='%m%d%Y')
    df = df[df['TRANSACTION_DATE'].dt.year == 2010]
    del df['TRANSACTION_DATE']
    grp = df.groupby('BUYER_ZIP')
    merge = pd.merge(merge, grp.sum(), how='left', on='BUYER_ZIP')
    merge = merge.fillna(0)
    merge['DOSAGE_UNIT_x'] = merge['DOSAGE_UNIT_x'] + merge['DOSAGE_UNIT_y']
    merge = merge.rename(columns={'DOSAGE_UNIT_x': 'DOSAGE_UNIT'})
    del merge['DOSAGE_UNIT_y']
merge = merge.rename(columns={'BUYER_ZIP': 'Zip Code', 'DOSAGE_UNIT': 'Pills'})
merge = merge.sort_values(by=['Pills'], ascending=False)
merge['Pills'] = merge['Pills'].astype('int32')
merge.to_csv('pills-per-zipcode.csv', index=False)